In [ ]:
import numpy as np
import sys

sys.path.append("/kesemi/g2opy/lib/")

from pathlib import Path

from kesemi import PoseOptimizer
from kesemi.image_providers import QueryImage, TrainImage
from kesemi.orb_alignmnet import ORBExtractor, ORBMatcher
from kesemi.superpoint_alignment import SuperPointExtractor, SuperPointMatcher

In [ ]:
keypoints_type = "ORB"  # OR "SP"
train_intrinsics = np.asarray([[481.20, 0, 319.50],
                               [0, -480.00, 239.50],
                               [0, 0, 1]])
# Replace if query and training images are taken from different cameras
query_intrinsics = train_intrinsics
depth_scale = 5000

path_to_train_depth_image = Path("test_images/depth/0150.png")
path_to_train_color_image = Path("test_images/rgb/0150.png")
path_to_query_color_image = Path("test_images/rgb/0200.png")

In [ ]:
if keypoints_type == "ORB":
    extractor = ORBExtractor(nfeatures=1000)
    matcher = ORBMatcher(ratio_threshold=0.7)
elif keypoints_type == "SP":
    extractor = SuperPointExtractor("superpoint_v1.pth")
    matcher = SuperPointMatcher(nn_thresh=0.7)
else:
    raise ValueError("Keypoints type should be ORB or SuperPoint")

# On some scenes, the results are better if the threshold is much higher (e.g., 20000)
pose_optimizer = PoseOptimizer(query_intrinsics, reproj_tresh=30)

In [ ]:
train_image = TrainImage(
    path_to_train_depth_image,
    path_to_train_color_image,
    train_intrinsics,
    extractor,
    depth_scale,
)
query_image = QueryImage(path_to_query_color_image, extractor)

matches = matcher.match(train_image.descs, query_image.descs)

kpts_obs = []
keyframe_3d_points = []
for matched_keyframe_kp_i, matched_query_kp_i in matches:
    keyframe_3d_points.append(train_image.kpts_3d[matched_keyframe_kp_i])
    kpts_obs.append(query_image.kpts[matched_query_kp_i])
T = pose_optimizer.optimize(keyframe_3d_points, kpts_obs)

In [ ]:
np.savetxt("T.txt", T)